# The Game of Life

In this notebook, we want to simulate the Game of Life

## A few informations about JuPyteR

### You can use maths in the LaTeX way

Math in text $\alpha = 5$ is possible as well as standalone:

$$
\Psi = \int_{0}^{\infty} \omega(t)dt
$$


### You can use many environments

* list
* list


1. numbered list
2. tada


### Stuff from Internet

![](https://upload.wikimedia.org/wikipedia/commons/d/d8/Game_of_life_U.gif)


## Game of Life

Documentation about GoL is available [here on wikipedia](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)

The rule of the game is:

* Cells can be either dead (0) or alive (1)
* At every step, there is are two rules:
    * A birth rule: a dead cell becomes alive if it has 3 living neigbors
    * A survival rule! a living cell stays alive if it has 2 or 3 living neighbors
    
The rule can be summarized as `B3S23`    

In [88]:
# Matplotlib "notebook" backend is an interactive backend for notebooks
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import numba

## Let's code the GoL

### First, we need to compute the Moore neighbors

![](https://upload.wikimedia.org/wikipedia/commons/4/4d/Moore_neighborhood_with_cardinal_directions.svg)

In [20]:
# Starting cells
cells = np.random.randint(2, size=(10, 10))
cells

array([[1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 1, 0, 0, 0, 1]])

In [89]:
@numba.jit()
def count_neighbors(cells):
    """
    Counts the Moore neighbors of the cell matrix
    
    Inputs:
    * cells: NxM array-like
    
    Returns:
    * neighbors: NxM array-like 
    """
    Nr, Nc = cells.shape # Number of rows and columns
    neighbors = np.zeros_like(cells) # A matrix with the same shape and datatype as cells
    for r in range(Nr): # Loop on row
        for c in range(Nc) : # Loop over columns
            neighbors[r,c] = cells[max(0,r-1) : min(r+2, Nr), 
                                   max(0,c-1) : min(c+2, Nc)].sum() - cells[r, c] 

    return neighbors

count_neighbors(cells)

array([[2, 2, 3, 1, 4, 2, 4, 2, 2, 1],
       [4, 3, 4, 2, 4, 4, 7, 5, 6, 3],
       [2, 2, 3, 2, 3, 4, 5, 5, 5, 3],
       [1, 3, 3, 3, 2, 4, 5, 7, 5, 3],
       [1, 3, 3, 2, 2, 4, 4, 5, 4, 3],
       [2, 2, 5, 3, 3, 2, 3, 4, 1, 1],
       [3, 2, 3, 0, 3, 4, 4, 3, 2, 2],
       [1, 3, 3, 3, 4, 4, 4, 4, 4, 2],
       [2, 3, 2, 4, 5, 6, 5, 3, 4, 3],
       [1, 1, 3, 3, 4, 4, 4, 3, 4, 2]])

In [90]:
# Let's have a look at the speed of our function
bigcells = np.random.randint(2, size=(200, 200))
%timeit count_neighbors(bigcells)

1.76 ms ± 86.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Apply the rule of GoL

In [97]:
B = [3] # Birth rule
S = [2,3] # Survival rule
neigh = count_neighbors(cells)
( (neigh == 3) & (cells == 0) )*1 # Logical and = &, logical or = | 

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0]])

In [94]:
True

True